# Battle Royale Environment Trainer
This notebook is for training Battle Royale agents. MADDPG is used for training the agents.

## Setup Environment Dependencies

In [46]:
import sys
from gym_unity.envs import UnityEnv

%matplotlib inline

print("Python version:")
print(sys.version)
print(sys.executable)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")

Python version:
3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
/Users/adhipradhana/anaconda3/envs/unity-battle-royale/bin/python


## Start Environment

In [47]:
# Environment name
# Remember to put battle royale environment configuration within the config folder
env_name = "environment/battle-royale"

env = UnityEnv(env_name, worker_id=1, use_visual=False, multiagent=True)

print(str(env))

INFO:mlagents_envs:Connected new brain:
PlayerBrain?team=0
INFO:gym_unity:4 agents within environment.


<UnityEnv instance>


## Examine Observation Space

In [48]:
# Examine observation space
observation = env.observation_space
print("Agent observation space type: {}".format(observation))

Agent observation space type: Box(42,)


## Examine Action Space

In [49]:
# Examine action space
action = env.action_space
print("Agent action space type: {}".format(action))

Agent action space type: Box(5,)


## Agents Training
This part shows agent training using MADDPG algoritm

### Setup Algorithm Dependencies

In [50]:
from datetime import datetime
import torch
import visdom
import numpy as np

from utils.MADDPG import MADDPG
from utils.RandomProcess import OUNoise

### Setup Algoritm Parameters

In [52]:
random_seed = 3677
n_states = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]
n_agents = env.number_agents
n_episode = 10000
max_steps = 1000
buffer_capacity = 1000000
batch_size = 1000
episodes_before_train = 100

### Setup MADDPG

In [53]:
# setup seed
torch.manual_seed(random_seed)
np.random.seed(random_seed)

maddpg = MADDPG(n_agents, n_states, n_actions, batch_size, buffer_capacity, episodes_before_train)
noise = OUNoise(env.action_space)

FloatTensor = torch.cuda.FloatTensor if maddpg.use_cuda else torch.FloatTensor

vis = visdom.Visdom(port=8097)

INFO:visdom:Visdom successfully connected to server


### MADDPG Training

In [58]:
win = None
current_time = str(datetime.now())
reward_record = []

print("Exploration begins...")
for i_episode in range(n_episode):
    # reset environment
    obs = env.reset()
    obs = np.stack(obs)
    noise.reset()
    
    # convert observation to tensor
    if isinstance(obs, np.ndarray):
        obs = torch.from_numpy(obs).float()
    
    total_reward = 0.0
    rr = np.zeros((n_agents,))
    for i_step in range(max_steps):
        obs = obs.type(FloatTensor)
        actions = maddpg.select_action(obs).data.cpu()
        actions = [noise.get_action(action) for action in actions.tolist()]
        
        obs_, reward, done, _ = env.step(actions)

        reward = torch.FloatTensor(reward).type(FloatTensor)
        obs_ = np.stack(obs_)
        obs_ = torch.from_numpy(obs_).float()
        if i_step != max_steps - 1:
            next_obs = obs_
        else:
            next_obs = None

        total_reward += reward.sum()
        rr += reward.cpu().numpy()
        maddpg.memory.push(obs.data, action, next_obs, reward)
        obs = next_obs

        c_loss, a_loss = maddpg.update_policy()
        
        # check if done
        if done[0]:
            break

    maddpg.episode_done += 1
    print("Episode: {}, reward = {}".format(i_episode, total_reward))
    reward_record.append(total_reward)
    
    if maddpg.episode_done == maddpg.episodes_before_train:
        print("Training begins...")
        print("MADDPG on Battle Royale")
              
    if win is None:
        win = vis.line(X=np.arange(i_episode, i_episode+1),
                       Y=np.array([
                           np.append(total_reward, rr)]),
                       opts=dict(
                           ylabel="Reward",
                           xlabel="Episode",
                           title="MADDPG on Battle Royale | " + \
                               "Agent: {} | ".format(n_agents) + \
                               "Time: {}\n".format(current_time),
                           legend=["Total"] +
                           ["Agent-".format(i) for i in range(n_agents)]))
    else:
        vis.line(X=np.array(
            [np.array(i_episode).repeat(n_agents+1)]),
                 Y=np.array([np.append(total_reward,rr)]),
                 win=win,
                 update="append")

Exploration begins...
Episode: 0, reward = 0.794999897480011
Episode: 1, reward = 0.6200000047683716
Episode: 2, reward = 0.5299999713897705
Episode: 3, reward = 0.8599998354911804
Episode: 4, reward = 0.6800000071525574


KeyboardInterrupt: 

## Close Environment

In [45]:
env.close()

INFO:mlagents_envs:Environment shut down with return code 0.
